In [29]:
import multiprocessing as mp # Paralelismo, concurrencia y aprovechar al maximo todos los procesadores en una maquina
import os
import linecache # Acceso aleatoria a lineas de texto
from functools import partial # permite que al llamar a un objeto este se comporte como funcion
print("Procesadores: {}".format(mp.cpu_count()))

Procesadores: 2


In [30]:
### data available from https://drive.google.com/drive/folders/12TutHqeblFfuc9jgl9V1W4wUsvpQCF5K?usp=sharing

!cp /content/drive/MyDrive/Grandes_Datos_Ulises_Enero2023/checkouts_seattle.zip .

^C


In [3]:
!unzip /content/checkouts_seattle.zip

Archive:  /content/checkouts_seattle.zip
  inflating: Checkouts by Title FAQs.pdf  
  inflating: checkouts-by-title.csv  
  inflating: socrata_metadata.json   


In [4]:
# the ls command shows a list of all the files in a directory
# -l is an option to show the list in large format
# -h is an option to show the sizes of the files in 'human readable' format
ls -lh 

total 9.4G
-rw-r--r-- 1 root root 7.2G Dec  3  2019  checkouts-by-title.csv
-rw-r--r-- 1 root root 363K Dec  3  2019 'Checkouts by Title FAQs.pdf'
-rw------- 1 root root 2.3G Jan 28 01:16  checkouts_seattle.zip
drwx------ 7 root root 4.0K Jan 28 01:11  drive/
drwxr-xr-x 1 root root 4.0K Jan 26 14:33  sample_data/
-rw-r--r-- 1 root root  16K Dec  3  2019  socrata_metadata.json


In [5]:
!tail checkouts-by-title.csv

Physical,Horizon,BOOK,2014,8,6,Fueled for adventure / adapted by Christine Peymani and Adam B. Murr ; illustrated by Carson Van Osten.,"Peymani, Christine","Automobiles Juvenile fiction, Automobile racing Juvenile fiction, Movie novels","Golden Books,",c2011.
Digital,OverDrive,EBOOK,2014,8,1,Why God Won't Go Away: Brain Science and the Biology of Belief,"Andrew Newberg, M.D.","Nonfiction, Religion & Spirituality, Science","Random House, Inc.",2012
Digital,OverDrive,EBOOK,2014,8,2,Criss Cross,Lynne Rae Perkins,"Historical Fiction, Young Adult Fiction",HarperCollins Publishers Inc.,2010
Physical,Horizon,BOOK,2014,8,1,The woman who shot Mussolini / Frances Stonor Saunders.,"Saunders, Frances Stonor","Gibson Violet, Mussolini Benito 1883 1945 Assassination attempt 1926 April 7, Assassins Biography, Catholic converts Biography, Aristocracy Social class Ireland Dublin Biography, Ashbourne Edward Gibson Baron 1837 1913 Family, Ireland History 1837 1901 Biography, Dublin Ireland Biography, Mus

In [6]:
#subsample of 1M of file
!head -n 1000000 checkouts-by-title.csv > checkouts10M.csv

In [28]:
%%time
# word count of the number of lines
!wc -l checkouts-by-title.csv

34892624 checkouts-by-title.csv
CPU times: user 230 ms, sys: 56.3 ms, total: 287 ms
Wall time: 32.8 s


In [8]:
%%time
!wc -l checkouts10M.csv

1000000 checkouts10M.csv
CPU times: user 12.2 ms, sys: 6.46 ms, total: 18.6 ms
Wall time: 816 ms


In [9]:
filename = "checkouts10M.csv"
lines = 1000000

In [10]:
#threads = mp.cpu_count() #threads disponibles en la máquina
threads = 4
num_lines = lines
split_size = num_lines // threads if num_lines // threads > 0 else 1 #Tamaños a procesar por cada procesador
parts = [x for x in range(0, num_lines, split_size)] #Donde inicia 
parts

[0, 250000, 500000, 750000]

In [11]:
def countWordsInTitles(filename, chunksize, start):
  wordCount = {}
  counter = 0
  print("Calling from {}".format(start))
  for current_line in range(start, (start+chunksize)+1):
    ln = linecache.getline(filename, current_line)
    try:
      words = ln.split(",")[6].split(" ")
      #print(words)
      for w in words:
        if w.upper() not in wordCount.keys():
          wordCount[w.upper()] = 0
        wordCount[w.upper()] += 1
    except Exception as e:
      print("null title {} ".format(str(e)))      
    counter += 1
    if counter % 1000000 == 0:
      print("From {start} in line {counter}".format(start=start, counter=counter))
  return wordCount

In [12]:
%%time
pool = mp.Pool(processes=threads) # distribuye los datos de entrada a traves de procesos
func = partial(countWordsInTitles, filename, split_size)
res = pool.map(func, parts)
pool.close() # liberacion de recursos
pool.join() # espera a la ejecucion de todos los procesos
# "res" entonces es una lista de diccionarios con las cuentas de palabras por cada worker.

Calling from 0Calling from 500000Calling from 250000

Calling from 750000

null title list index out of range 
CPU times: user 288 ms, sys: 140 ms, total: 428 ms
Wall time: 11.6 s


In [13]:
%%time
wordCount = {}
counter = 0
#f1 = open(filename, 'r')
for i in range(1,lines):
  ln = linecache.getline(filename, i)
  try:    
    words = ln.split(',')[6].split(" ")  
    for w in words:
      if w.upper() not in wordCount.keys():
        wordCount[w.upper()] = 0
      wordCount[w.upper()] += 1
  except Exception as e:
    print("null title {} ".format(str(e)))      
  counter += 1
  if counter % 1000000 == 0:
    print("From {start} in line {counter}".format(start=0, counter=counter))

CPU times: user 8.87 s, sys: 209 ms, total: 9.08 s
Wall time: 9.12 s


In [14]:
res[0]

{'TITLE': 31,
 "MCGRAW-HILL'S": 5,
 'DICTIONARY': 429,
 'OF': 52794,
 'AMERICAN': 3230,
 'SLANG': 23,
 'AND': 42285,
 'COLLOQUIAL': 14,
 'EXPRESSIONS': 20,
 '/': 77811,
 'RICHARD': 1299,
 'A.': 977,
 'SPEARS.': 15,
 '"EMMA': 1,
 'RED': 856,
 'MIDNIGHT': 252,
 'JUST': 512,
 'THE': 86240,
 'FINANCIAL': 225,
 'FACTS': 164,
 'HOW': 5058,
 'TO': 22238,
 'IDENTIFY': 30,
 'NUGGETS': 7,
 'USABLE': 6,
 'NEWS': 160,
 'MINIMIZE': 7,
 'INFORMATION': 129,
 'OVERLOAD': 8,
 'SINGLE': 108,
 'SHARD': 4,
 'PAUL': 1089,
 "KIRK'S": 2,
 'CHAMPIONSHIP': 22,
 'BARBECUE': 32,
 ':': 35084,
 'BBQ': 10,
 'YOUR': 6323,
 'WAY': 1398,
 'GREATNESS': 22,
 'WITH': 9313,
 '575': 2,
 "LIP-SMACKIN'": 2,
 'RECIPES': 1254,
 'FROM': 8345,
 'BARON': 25,
 'KIRK': 47,
 'BOB': 565,
 'LYON.': 9,
 'LIVE': 1206,
 'THIS': 839,
 'IS': 2546,
 'HOUSE': 1566,
 '"BANDIDO': 1,
 'OSCAR': 107,
 '""ZETA""': 1,
 'ACOSTA': 1,
 'CHICANO': 6,
 'EXPERIENCE': 220,
 'ILAN': 11,
 'STAVANS."': 1,
 'COLUMBIA': 307,
 'RIVER': 486,
 'COLLECTION': 1103,

In [15]:
#Eso fue el mapper, este es el reduce
#cuantos mappers teniamos?
len(res)

4

In [16]:
reduced = {} #Creamos el diccionario reducido
for i in res: #Para cada elemento del mapper
  for key in i.keys(): #Recorremos las llaves
    if key not in reduced.keys(): #si no existe la creamos con el primer valor que encuentra para esa llave
      reduced[key] = i[key] 
    else:
      reduced[key] = reduced[key] + i[key] #si existe, entonces le suma el valor

In [17]:
print("Total keys in first", len(res[0].keys()))
print("Total keys in second", len(res[1].keys()))

Total keys in first 111966
Total keys in second 118153


In [18]:
print("Total Keys reduced", len(reduced.keys()))

Total Keys reduced 240508


In [19]:
reduced.keys() #llaves, tomar ejemplo aleatorio


dict_keys(['TITLE', "MCGRAW-HILL'S", 'DICTIONARY', 'OF', 'AMERICAN', 'SLANG', 'AND', 'COLLOQUIAL', 'EXPRESSIONS', '/', 'RICHARD', 'A.', 'SPEARS.', '"EMMA', 'RED', 'MIDNIGHT', 'JUST', 'THE', 'FINANCIAL', 'FACTS', 'HOW', 'TO', 'IDENTIFY', 'NUGGETS', 'USABLE', 'NEWS', 'MINIMIZE', 'INFORMATION', 'OVERLOAD', 'SINGLE', 'SHARD', 'PAUL', "KIRK'S", 'CHAMPIONSHIP', 'BARBECUE', ':', 'BBQ', 'YOUR', 'WAY', 'GREATNESS', 'WITH', '575', "LIP-SMACKIN'", 'RECIPES', 'FROM', 'BARON', 'KIRK', 'BOB', 'LYON.', 'LIVE', 'THIS', 'IS', 'HOUSE', '"BANDIDO', 'OSCAR', '""ZETA""', 'ACOSTA', 'CHICANO', 'EXPERIENCE', 'ILAN', 'STAVANS."', 'COLUMBIA', 'RIVER', 'COLLECTION', 'JOURNEY', 'STORIES', 'MIGRATION', 'CYNTHIA', 'RYLANT', ';', 'ILLUSTRATED', 'BY', 'LAMBERT', 'DAVIS.', 'I', 'WANNA', 'TAKE', 'ME', 'A', 'PICTURE', 'TEACHING', 'PHOTOGRAPHY', 'WRITING', 'CHILDREN', 'GOOGIE', 'FIFTIES', 'COFFEE', 'SHOP', 'ARCHITECTURE', 'ALAN', 'HESS.', 'LAZARUS', 'HEART', 'SOME', 'BEST', 'BIAŁA', 'BLUZKA', 'AGNIESZKA', 'OSIECKA.', 'RE

In [20]:
reduced["AMERICAN"]


11719

In [21]:
res[0]["AMERICAN"]

3230

In [22]:

res[1]["AMERICAN"]

3064

In [23]:

res[0]["AMERICAN"] + res[1]["AMERICAN"]

6294

In [24]:
res[2]["AMERICAN"]

2802

In [25]:
res[0]["AMERICAN"] + res[1]["AMERICAN"] + res[2]["AMERICAN"]

9096

In [26]:
res[3]["AMERICAN"]

2623

In [27]:
res[0]["AMERICAN"] + res[1]["AMERICAN"] + res[2]["AMERICAN"] + res[3]["AMERICAN"]

11719